## 네이버 AI어플리케이션 기반 뉴스 핵심 요약 및 분석


활용엔진 
  - 번역 : 파파고 
  - 요약 : 클로바 
  - 감성분석 : 클로바     
  
업데이트 내역 
  - 2022-05-18 (hanhoman)
  
분석 플랫폼
  - 네이버 클라우드

In [1]:
import sys
import requests
import json

In [4]:
client_id = "HERE'S YOUR KEY"
client_secret = "HERE'S YOUR KEY"

In [2]:
def ai_news(title, content, lan="ko", printout=True, get_return=True):
    """
    title : 뉴스 제목 (필수)
    content : 뉴스 원문 
    lan : 언어 (ko : 한국어, en: 영문)
    printout : 결과물 출력 여부 (기본값 True)
    get_return : 결과물 리턴 여부 (기본값 True) 
      리턴값 : 뉴스원문, 핵심요약(3개), 감성분석결과(긍정/부정/중립), 긍정점수, 중립점수, 부정점수    
    """

    # 번역 (파파고) 
    def get_translate(text):        
        data = {'text'   : text,
                'source' : 'en',
                'target' : 'ko'}

        url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
        
        header = {"X-NCP-APIGW-API-KEY-ID":client_id,
                  "X-NCP-APIGW-API-KEY":client_secret}

        response = requests.post(url, headers=header, data=data)
        rescode = response.status_code

        if(rescode==200):
            send_data = response.json()
            trans_data = (send_data['message']['result']['translatedText'])
            if printout:
                print("=== 한글번역본 ===")
                print(trans_data) 
            return trans_data
        else:
            print("Error Code:" , rescode)
            
    # 핵심요약 (클로바)
    def get_summary(title, content, 
                    language = "ko", model = "news", tone = "2", summaryCount = "3"):    
        """
        출처 : https://bit.ly/3wjSoP8
        
        파라미터 :
        language = "ko"     # Language of document (ko, ja )
        model = "news"      # Model used for summaries (general, news)
        tone = "2"          # Converts the tone of the summarized result. (0, 1, 2, 3)
        summaryCount = "3"  # This is the number of sentences for the summarized document.    
        """
        
        url = "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize"
        
        headers = {"X-NCP-APIGW-API-KEY-ID":client_id,
                   "X-NCP-APIGW-API-KEY":client_secret,
                   "Content-Type": "application/json"}
        
        data = {
            "document": {
            "title": title,
            "content" : content
            },
            "option": {
            "language": language,
            "model": model,
            "tone": tone,
            "summaryCount" : summaryCount
            }
        }
        
        response = requests.post(url, data=json.dumps(data), headers=headers)
        rescode = response.status_code
        
        if(rescode == 200):
            res = json.loads(response.text)['summary'].split('\n')  # 요약별 리스트 반환 
            if printout:
                print("\n=== 핵심요약 ===")
                for i, txt in enumerate(res):
                    print("#"+str(i+1)+'.', txt)
            return res
        else:
            print("Error : " + response.text)
            return None

    # 감성분석  (클로바)
    def get_sentiment(content):

        url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

        headers = {
            "X-NCP-APIGW-API-KEY-ID": client_id,
            "X-NCP-APIGW-API-KEY": client_secret,
            "Content-Type": "application/json"
        }

        data = {
          "content": content
        }

        response = requests.post(url, data=json.dumps(data), headers=headers)
        rescode = response.status_code

        if(rescode == 200):        
            sentiment = json.loads(response.text)['document']['sentiment'] 
            pos_score = json.loads(response.text)['document']['confidence']['positive']
            neg_score = json.loads(response.text)['document']['confidence']['negative']
            neu_score = json.loads(response.text)['document']['confidence']['neutral']
            if printout:
                print("\n=== 감성분석 ===")
                print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
            return sentiment, pos_score, neu_score, neg_score
            
        else:
            print("Error : " + response.text)
            return None            
            


    if lan=='en':  # 영문의 경우 한글로 번역 
        content = get_translate(content)
    content_summary = get_summary(title, content)
    sentiment, pos_score, neu_score, neg_score = get_sentiment(content[:500]) # 500글자까지만 보내기
       
    if get_return:
        return content, content_summary, sentiment, pos_score, neu_score, neg_score   
    else:
        return None

---
### 검증 1. 영문 

In [5]:
title = "Stock futures rise to extend gains after Powell comments"

content = """U.S. stock futures opened higher Tuesday afternoon after rallying during the regular trading day, as investors took in reassurances from Federal Reserve Chair Jerome Powell that the central bank was set on using its policies to bring down inflation still running at multi-decade highs.
Contracts on the S&P 500 edged higher. The blue-chip index ended Tuesday's regular trading day higher by 2% to settle at at 4,088.85. Technology and growth stocks that had been beaten down over the past month recovered some losses, sending the Nasdaq higher by 2.8%. And the cyclical small-cap Russell 2000 also shook off some recent losses, climbing 3.2%.
The market moves Tuesday came following a couple of solid reports on U.S. economic activity, showing both consumer spending and manufacturing production were holding up strongly. U.S. retail sales grew at a 0.9% rate in April after a sharply upwardly revised 1.4% monthly rise in March, suggesting consumers were continuing to spend even as consumer prices have climbed at the fastest rate since the 1980s. The latest print on U.S. industrial production also exceeded estimates with a jump of 1.1% last month, or more than double the expected rise.
"""

ai_news(title, content, lan="en", printout=True, get_return=False)

=== 한글번역본 ===
미국 증시는 제롬 파월 연방준비제도이사회(FRB) 의장으로부터 중앙은행이 수십년 만에 최고치를 경신하고 있는 인플레이션을 억제하기 위해 정책을 사용할 것이라는 확신을 얻으면서 상승세로 장을 마쳤다.
S&P 500지수는 소폭 상승했다. 우량주 지수는 화요일의 정규 거래일을 2% 상승한 4,088.85로 마감했다. 지난 한 달 동안 내리막길을 걷던 기술주와 성장주들이 일부 손실을 회복하며 나스닥 지수를 2.8% 상승시켰다. 그리고 순환형 소형주 러셀 2000도 3.2% 상승하며 최근 손실을 일부 상쇄했다.
화요일 시장의 움직임은 소비지출과 제조업 생산이 모두 견조하다는 것을 보여주는 미국의 경제활동에 대한 두 가지 확실한 보고에 이어 나왔다. 미국의 소매판매는 3월에 월별 1.4%의 대폭 상향 조정된 후 4월에 0.9%의 비율로 증가했는데, 이는 소비자가격이 1980년대 이후 가장 빠른 속도로 상승했음에도 불구하고 소비를 계속하고 있음을 시사한다. 미국 산업생산에 대한 최근 인쇄물도 지난달 예상치의 2배 이상인 1.1% 급등하며 예상치를 웃돌았다.

=== 핵심요약 ===
#1. 미국 증시는 제롬 파월 연방준비제도이사회(FRB) 의장으로부터 중앙은행이 수십년 만에 최고치를 경신하고 있는 인플레이션을 억제하기 위해 정책을 사용할 것이라는 확신을 얻으면서 상승세로 장을 마쳤습니다.
#2. 화요일 시장의 움직임은 소비지출과 제조업 생산이 모두 견조하다는 것을 보여주는 미국의 경제활동에 대한 두 가지 확실한 보고에 이어 나왔습니다.
#3. 미국의 소매판매는 3월에 월별 1.4%의 대폭 상향 조정된 후 4월에 0.9%의 비율로 증가했는데, 이는 소비자가격이 1980년대 이후 가장 빠른 속도로 상승했음에도 불구하고 소비를 계속하고 있음을 시사합니다.

=== 감성분석 ===
판정: positive
점수: 긍정 85.72951, 중립 14.243666, 부정 0.026828866


---
### 검증 2. 한글

In [6]:
title = "[클릭 e종목] 셀트리온의 실적 '냉각'…봄이 올까"

content = """하나금융투자는 13일 셀트리온이 올해 1분기 간 비용 상승에 따라 시장 기대치에 못 미치는 실적을 거뒀다며 목표주가를 21만원으로 낮췄다.
셀트리온은 올해 1분기간 매출액 5506억원을 달성했다. 이는 전년 같은 기간 대비 20.5% 늘어난 수치다. 반면, 영업이익은 같은 기간 -34.5% 줄어든 1423억원을 기록했다. 컨센서스인 매출액 5265억원, 영업이익 1979억원에 못 미치는 수준이다.
램시마 IV와 셀트리온 USA 의 진단키트가 매출액을 끌어올렸다. 하지만 상대적으로 수익성이 낮은 램시마 IV와 진단키트의 비중 확대와 렉키로나 관련 재고 자산 상각, 코로나19 흡입제형 연구개발비 등에 따라 영업이익률은 전년 동기 45.4% 대비 낮아진 25.8%를 기록했다
하나금투는 올해 셀트리온이 매출액 2조2095억원(전년 대비 15.6%), 영업이익 7210억원(전년 대비 -4.2%)을 달성할 것으로 전망했다.
박재경 하나금투 연구원은 "셀트리온의 현재 향후 12개월 간 주가수익비율(PER)은 31.2배 수준으로 2019년도의 저점에 가까워졌다"며 "추가적인 하락은 제한적"이라고 내다봤다.
"""

ai_news(title, content, lan="ko", printout=True, get_return=False)


=== 핵심요약 ===
#1. 하나금융투자는 13일 셀트리온이 올해 1분기 간 비용 상승에 따라 시장 기대치에 못 미치는 실적을 거뒀다며 목표주가를 21만원으로 낮췄습니다.
#2. 하나금투는 올해 셀트리온이 매출액 2조2095억원(전년 대비 15.6%), 영업이익 7210억원(전년 대비 -4.2%)을 달성할 것으로 전망했습니다.
#3. 박재경 하나금투 연구원은 "셀트리온의 현재 향후 12개월 간 주가수익비율(PER)은 31.2배 수준으로 2019년도의 저점에 가까워졌다"며 "추가적인 하락은 제한적"이라고 내다봤습니다.

=== 감성분석 ===
판정: negative
점수: 긍정 16.641035, 중립 6.448433, 부정 76.91053
